In [2]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [3]:
from labeling.upload import ETLProcessor, get_spark, get_batch_intervals, sampling
from omegaconf import OmegaConf

In [4]:
config = OmegaConf.load("config/local_full.yaml")

In [5]:
spark = get_spark()

In [7]:
from labeling.upload import get_batch_intervals

vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
combine_win_size = config.hp.etl.combine_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size

for bs, be in get_batch_intervals(
    start_block=7063372,
    global_end_block=7065370,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be)

7062873 7064871
7064313 7065351


In [8]:
7064871 - 7062873 + 1

1999

In [9]:
df = sampling(spark, config, 7062873, 7064871)

Successfully queried data from database


In [12]:
df.groupBy("block_height").count().count()

1999

In [11]:
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
------------------------------------------------
Sucessfully combine data
------------------------------------------------
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [13]:
dff.printSchema()

root
 |-- operator_address: string (nullable = true)
 |-- score: double (nullable = true)
 |-- voting_power_score: double (nullable = true)
 |-- tokens_proportion: decimal(38,10) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,4) (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- delegator_shares: decimal(38,4) (nullable = true)
 |-- tokens: decimal(38,4) (nullable = true)
 |-- commission_score: double (nullable = true)
 |-- block_height: integer (nullable = true)
 |-- vote_propose_score: double (nullable = true)
 |-- label: double (nullable = true)



In [14]:
dff.groupBy("block_height").count().count()

72

In [17]:
dp = dff.groupBy("block_height").count().pandas_api()

In [19]:
d = dp.values

d:\Apps\MiniConda3\envs\py38\lib\site-packages\pyspark\pandas\generic.py:647: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
d:\Apps\MiniConda3\envs\py38\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [21]:
bh = d[:, 0]

In [23]:
bh.sort()

In [24]:
import numpy as np
from collections import Counter

cnt = Counter(np.diff(bh))
cnt

Counter({20: 71})

In [25]:
bh[0], bh[-1]

(7063372, 7064792)

In [ ]:
7063372, 7064792
7062873, 7064871

In [28]:
7063372 - 7062873 + 1

500

In [30]:
7064871 - 7064792 + 1

80